In [1]:
%load_ext autoreload
import os

import numpy as np

%autoreload 2
import zcu_tools.notebook.analysis.design as zd
import zcu_tools.notebook.analysis.plot as zp

In [2]:
qub_name = "Design1"

os.makedirs(f"../../result/{qub_name}/image", exist_ok=True)
os.makedirs(f"../../result/{qub_name}/web", exist_ok=True)

# Scan params

In [3]:
EJb = (2.0, 7.5)
EC = 0.8
# EC = 1.4
ELb = (0.35, 1.0)

flx = 0.5
# r_f = 7.2
r_f = 7.52994
g = 0.1

In [42]:
Temp = 113e-3
Q_cap = 4.0e5
Q_ind = 1.7e7
x_qp = 1.5e-6

noise_channels = [
    ("t1_capacitive", dict(Q_cap=Q_cap)),
    ("t1_inductive", dict(Q_ind=Q_ind)),
    ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
]

avoid_freqs = [r_f, 2 * r_f]


params_table = zd.generate_params_table(EJb, EC, ELb, flx)

In [ ]:
zd.calculate_esys(params_table)
zd.calculate_f01(params_table)
zd.calculate_m01(params_table)
zd.calculate_t1(params_table, noise_channels, Temp)
zd.calculate_dipersive_shift(params_table, g=g, r_f=r_f)
zd.avoid_collision(params_table, avoid_freqs, threshold=0.2)
zd.avoid_low_f01(params_table, f01_threshold=0.1)
zd.avoid_low_m01(params_table, m01_threshold=0.05)
params_table.drop("esys", axis=1)

In [ ]:
fig = zd.plot_scan_results(params_table)
fig.update_layout(
    title=", ".join(
        ", ".join(f"{name} = {value:.1e}" for name, value in p_dict.items())
        for _, p_dict in noise_channels
    ),
    title_x=0.51,
)

best_params = zd.annotate_best_point(fig, params_table)
# zd.add_real_sample(fig, "Q12_2D/Q4", noise_channels=noise_channels, Temp=Temp, flx=flx)

fig.show()

In [39]:
save_name = f"t1vsM01_EC{EC:.2f}_rf{r_f:.2f}"
fig.write_html(f"../../result/{qub_name}/web/{save_name}.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/{save_name}.png", format="png")

In [ ]:
# best_params = 5.95, 0.8, 0.95

flxs = np.linspace(0.0, 1.0, 1000)
best_params

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(10) if j > i]

fig = zp.plot_transitions(best_params, flxs, show_idxs, ref_freqs=avoid_freqs)
fig.update_yaxes(range=(0.0, 15.0))
fig.show()

In [10]:
save_name = f"f01_EC{EC:.2f}_rf{r_f:.2f}"
fig.write_html(f"../../result/{qub_name}/web/{save_name}.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/{save_name}.png", format="png")

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(3) if j > i]

fig = zp.plot_matrix_elements(best_params, flxs, show_idxs)
fig.show()

In [12]:
save_name = f"Matrix_EC{EC:.2f}_rf{r_f:.2f}"
fig.write_html(f"../../result/{qub_name}/web/{save_name}.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/{save_name}.png", format="png")

In [ ]:
fig = zp.plot_dispersive_shift(best_params, flxs, r_f=r_f, g=g)
fig.update_yaxes(range=(r_f - 0.01, r_f + 0.01))
fig.show()

In [14]:
save_name = f"Chi_EC{EC:.2f}_rf{r_f:.2f}"
fig.write_html(f"../../result/{qub_name}/web/{save_name}.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/{save_name}.png", format="png")

In [ ]:
# Temp = 60e-3
# Q_cap = 1.0e5
# Q_ind = 1.0e7
# x_qp = 1.0e-8

fig = zp.plot_t1s(
    best_params,
    flxs,
    noise_channels=[
        ("t1_capacitive", dict(Q_cap=Q_cap)),
        # ("t1_inductive", dict(Q_ind=Q_ind)),
        # ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
    ],
    Temp=Temp,
)
title1 = f"EJ/EC/EL = {best_params[0]:.3f}/{best_params[1]:.3f}/{best_params[2]:.3f}"
title2 = ", ".join(
    ", ".join(f"{name} = {value:.1e}" for name, value in p_dict.items())
    for _, p_dict in noise_channels
)
fig.update_layout(
    title=title1 + "<br>" + title2,
    title_x=0.515,
)
fig.show()

In [41]:
save_name = f"T1_EC{EC:.2f}_rf{r_f:.2f}"
fig.write_html(f"../../result/{qub_name}/web/{save_name}.html", include_plotlyjs="cdn")
fig.write_image(f"../../result/{qub_name}/image/{save_name}.png", format="png")